In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

In [3]:
# 'best' in 'best1'

In [4]:
# import torch
# torch.ones(1,2).cuda()

In [5]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random
import time

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

K = 2
STITCH_BATCH_SIZE = 96 # todo study the effect
MAX_DEPTH = 10
THRESOULD = 0.8
TOTAL_THRESOULD = 0.8

RESULT_NAME = f"{int(time.time())}_result_food101_CKA_BS_{STITCH_BATCH_SIZE}_MD_{MAX_DEPTH}_T_{THRESOULD}_TT_{TOTAL_THRESOULD}_K_{K}"

EVAL_BATCH_SIZE = 64


# data_score = data_score.numpy()

In [6]:
# from stitchnet.stitchonnx.utils import load_hf_train_val_dset_with_test_split

# dataset_train, dataset_val = load_hf_train_val_dset_with_test_split('food101',train='train', val='validation',label="label",num_train=10000, num_val=1000)

# dataset_train.save_to_disk('./_data/food101/train')
# dataset_val.save_to_disk('./_data/food101/validation')

In [7]:
from datasets import load_from_disk, load_dataset

In [8]:
dataset_train = load_from_disk('./_data/food101/train', keep_in_memory=True)
dataset_val = load_from_disk('./_data/food101/validation', keep_in_memory=True)

In [9]:
# dataset_train.features['label'].int2str(0)

In [10]:
selected_labels = [0,1,2,3,4]
dataset_train = dataset_train.filter(lambda example: example['label'] in selected_labels)

Filter:   0%|          | 0/9998 [00:00<?, ? examples/s]

In [11]:
# dataset_train

In [12]:
dataset_val = dataset_val.filter(lambda example: example['label'] in selected_labels)

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
dataset_train

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 483
})

In [14]:
dataset_val

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 49
})

In [15]:
# dataset_train = dataset_train.with_format(type='torch')
# dataset_val = dataset_val.with_format(type='torch')


In [16]:
# dataset.save_to_disk('./_data/food101')

In [17]:
# def collate(items):
#     inps = []
#     tgts = []
#     for item in items:
#         inps.append(item['pixel_values'])
#         tgts.append(item['label'])
#     return torch.stack(inps,0), torch.stack(tgts,0)
    
# # dl_score = load_dl(dataset_train, batch_size=STITCH_BATCH_SIZE, shuffle=False, num_workers=1)

In [18]:
# STITCH_BATCH_SIZE

In [19]:
# print(np.stack(x['pixel_values']).shape)

In [20]:
# dataset_train

In [21]:
# torch.vstack(dataset_train['pixel_values']).shape

In [22]:
# from transformers import AutoProcessor
from tqdm import tqdm
# p = AutoProcessor.from_pretrained('microsoft/resnet-50')
inps = []
tgts = []
# for x in tqdm(dataset_train.take(STITCH_BATCH_SIZE)):
for x in tqdm(dataset_train.select(range(STITCH_BATCH_SIZE))):
    try:
        inps += x['pixel_values']
        # inps += p(x['image'])['pixel_values']
        tgts += [x['label']]
    except:
        pass
data_score = np.stack(inps)
data_score.shape

100%|██████████| 96/96 [00:13<00:00,  7.01it/s]


(96, 3, 224, 224)

In [23]:
# from datasets import load_dataset
# dataset = load_dataset('food101', streaming=False)

In [24]:
# dataset['validation']['label']

In [25]:
# dataset[]

In [26]:
# from transformers import AutoProcessor
# p = AutoProcessor.from_pretrained('microsoft/resnet-50')
# def processor_function(examples):
#     try:
#         r = p(examples["image"])
#         return r
#     except:
#         return {'pixel_values': None}

In [27]:
# dataset = dataset.map(processor_function, 
#             batched=False,writer_batch_size=30000,keep_in_memory=False).filter(lambda x:x['pixel_values'] is not None, 
#                batched=False,writer_batch_size=30000,keep_in_memory=False)

In [28]:
# dataset = dataset.remove_columns('image')

In [29]:
# dataset.save_to_disk('./_data/food101')

In [30]:
# dataset["train"][0]

In [31]:
# dir(dataset)

In [32]:
# dataset["validation"]

In [33]:
# help(task_evaluator.compute)

In [34]:
# ds = dataset["validation"].select([1,2,3]).map(processor_function, 
#             batched=False,writer_batch_size=30000,keep_in_memory=False)

In [35]:
# ds = ds.remove_columns(['image'])

In [36]:
# ds = dataset["validation"].select([1,2,3])

In [37]:
# np.array(ds[0]['pixel_values']).shape

In [38]:
# import numpy as np
# np.array(ds['pixel_values'][0]).shape

In [39]:
# dataset

In [40]:
# ds.features['label']

In [41]:
# from PIL import Image

In [42]:
# x = Image.open("./PNG_Test.png")

In [43]:
# from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
# ds = load_dl([[x],[x],[x]])

In [44]:
# next(iter(ds))

In [45]:
# from evaluate import evaluator
# task_evaluator = evaluator("image-classification")

# class EvalPipeline:
#     def __init__(self, net):
#         self.task = "image-classification"
#         self.net = net
#     def __call__(self, inps, **kwargs):
#         # inps
#         # return []
#         # inps = [1]
#         print('inps', len(inps), inps[0])
#         result = [[{
#             "score": 0.1,
#             "label": 6
#         }] for p in inps]
#         # print('result', result)
#         return result
# net = None
# pipe = EvalPipeline(net)
# ds = dataset['validation']
# # label_mapping = {}
# # for name in ds.features['label'].names:
# #     label_mapping[name] = ds.features['label'].str2int(name)
# task_evaluator.compute(pipe, 
#                        data=ds,
#                        metric="accuracy")

In [46]:
# ds

In [47]:
# label_mapping

In [48]:
# max([{"score": 1}, {"score": 2}], key=lambda x: x["score"])

In [49]:
# dataset['train']

In [50]:
# dataset['train'].features['label'].num_classes

In [51]:
# dataset['train'].features['label'].int2str

In [52]:
# dir(dataset['train'].features['label'])

In [53]:
# dataset_val.info

In [54]:
# print(d['label'].shape)

In [55]:
# data_score.shape

In [56]:
# data_score, t = next(iter(dl_score))
# print(data_score,t)

In [57]:
# t.shape

In [58]:
# data_score.shape

In [59]:
# [None] is not [None]

In [60]:
# dataset_train[0]

In [61]:
# dataset_val[0]

In [62]:
# dataset_train[0]['pixel_values'].shape

In [63]:
# import numpy as np

# arr = [t for (x,t) in dataset_train]
# # Get unique values and their counts
# unique_values, counts = np.unique(arr, return_counts=True)
# # Print the results
# for value, count in zip(unique_values, counts):
#     print(f"TRAIN: {value} occurs {count} times.")
    
# arr = [t for (x,t) in dataset_val]
# # Get unique values and their counts
# unique_values, counts = np.unique(arr, return_counts=True)
# # Print the results
# for value, count in zip(unique_values, counts):
#     print(f"VAL: {value} occurs {count} times.")

In [64]:
# next(iter(dataset_train))

In [65]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [66]:
# for dataItem in tqdm(dataset_train):
#     print(dataItem['pixel_values'].shape)
#     break

In [67]:
from tqdm import tqdm
from stitchnet.stitchonnx.report import Report, ReportPlants, ReportKNNHFDataset
import traceback
scoreMapper = ScoreMapper(nets, data_score, scoring_method='CKA')
with ReportKNNHFDataset(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'a') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False, K=K)
    for i,(s,net) in enumerate(generator):
        try:
            netname = f"_results/{RESULT_NAME}/net{k:03}"
            report.evaluate(nets, net, netname, s, dataset_val, dataset_train)
            net.save(netname)
            k += 1
        except Exception as e:
            print('ERROR', e)
            traceback.print_exc()
            pass

(96, 3, 224, 224)
(96, 64, 27, 27)
(96, 192, 13, 13)
(96, 384, 13, 13)
(96, 256, 13, 13)
(96, 9216)
(96, 4096)
(96, 4096)
current depth: 1
(96, 3, 224, 224)
(96, 3, 224, 224)
(96, 3, 224, 224)
(96, 16, 56, 56)(96, 3, 224, 224)

(96, 64, 56, 56)(96, 16, 56, 56)

(96, 72, 28, 28)(96, 64, 224, 224)(96, 256, 56, 56)(96, 256, 56, 56)



(96, 64, 112, 112)
(96, 24, 28, 28)
(96, 96, 14, 14)(96, 512, 28, 28)(96, 128, 112, 112)
(96, 512, 28, 28)


(96, 128, 56, 56)
(96, 40, 14, 14)
(96, 256, 56, 56)
(96, 1024, 14, 14)
(96, 120, 14, 14)
(96, 1024, 14, 14)
(96, 256, 56, 56)(96, 2048)

(96, 48, 14, 14)(96, 256, 28, 28)

(96, 512, 28, 28)
(96, 1024)
(96, 288, 7, 7)
(96, 512, 28, 28)(96, 96, 7, 7)

(96, 576)
(96, 1024)(96, 512, 14, 14)

(96, 512, 14, 14)
(96, 512, 14, 14)
(96, 25088)
(96, 4096)
(96, 4096)
potential next fragments: 2
potential next fragments after thresholding of 0.8: 1 ['1.0']
curr <stitchnet.stitchonnx.utils.Fragment object at 0x7eff42556e90>
nextf <stitchnet.stitchonnx.utils.Fragm

100%|██████████| 16/16 [00:00<00:00, 71.21it/s]


(483, 1000) (483,)


 50%|█████     | 1/2 [00:03<00:03,  3.13s/it]

(32, 3, 224, 224)


...

 50%|█████     | 1/2 [00:03<00:03,  3.06s/it]

(32, 3, 224, 224)


100%|██████████| 2/2 [00:04<00:00,  2.42s/it]

(17, 3, 224, 224)
accuracy 0.6326530612244898


saving to _results/1689175585_result_food101_CKA_BS_96_MD_10_T_0.8_TT_0.8_K_2/net010
current depth: 1
potential next fragments: 2
potential next fragments after thresholding of 0.8: 1 ['1.0']
curr <stitchnet.stitchonnx.utils.Fragment object at 0x7efe5cb716f0>
nextf <stitchnet.stitchonnx.utils.Fragment object at 0x7efe5cb71120>
totalscore 0.9999998807907104
diff sampled tensor(0.)
epoch 0 loss 1.664057400386723e-05 torch.Size([4816896, 64]) torch.Size([4816896, 64])
(96, 3, 224, 224)
current depth: 2
potential next fragments: 2
potential next fragments after thresholding of 0.8: 1 ['1.0']
curr <stitchnet.stitchonnx.utils.Fragment object at 0x7efe3c0ebfd0>
nextf <stitchnet.stitchonnx.utils.Fragment object at 0x7efe5cb710c0>
totalscore 0.9999998211860728
diff sampled tensor(25.7547)
epoch 0 loss 2.0189481181684832e-05 torch.Size([1204224, 64]) torch.Size([1204224, 64])
(96, 3, 224, 224)
current depth: 3
potential next fragments: 2
potential next fragments after thresholding of 0.8: 1 ['1.

In [68]:
# from torch.utils.data import TensorDataset



In [69]:
# for x in dataset_train:
#     print(x)
#     break

In [70]:
# for x in dl:
#     print(x[0].squeeze(1).shape, x[1].shape)
#     break

In [71]:
# len(dataset_train)

In [72]:
# report.evaluate(nets, net, netname, s, dataset_val, dataset_train)

In [73]:
# import onnxruntime as ort
# ort.get_available_providers()

In [74]:
# for x in dataset_train:
#     print(x)
#     break

In [75]:
# from stitchnet.stitchonnx.utils import accuracy_score_net_plants

In [76]:
# torch.tensor([1,2]).numpy().tobytes()

In [77]:
# for x,t in dataset_val:
#     print(t)

In [78]:
# dataset_val[0]

In [79]:
# accuracy_score_net_plants(net, dataset_val, dataset_train)

In [80]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(np.ones([100,10]), np.ones([100,1]))


In [81]:
# y = knn.predict(np.ones([1,10]))
# y

In [84]:
# dataset_val

In [83]:
print('DONE')

DONE
